<a href="https://www.kaggle.com/code/shravankumar147/resume-ai-langchainrag?scriptVersionId=226845999" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Install required packages (run this cell if not already installed)
!pip install -q torch transformers accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl datasets pypdf langchain-community langchain-huggingface ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 92.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.2/294.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy
from ragatouille import RAGPretrainedModel
from typing import Optional, List, Tuple
import pandas as pd

##########################
# 1. Load Resume Dataset
##########################

# Path to the data directory containing resume PDFs
# Assuming the following structure:
# data/
#  ├── HR/
#  │   ├── resume1.pdf
#  │   └── resume2.pdf
#  ├── Designer/
#  │   ├── resume3.pdf
#  │   └── resume4.pdf
#  └── ...
DATA_DIR = "/kaggle/input/resume-dataset/data/data"
CSV_PATH = "/kaggle/input/resume-dataset/Resume/Resume.csv"  # Path to the CSV file with resume metadata

# Load CSV data for metadata
try:
    resume_df = pd.read_csv(CSV_PATH)
    print(f"Loaded metadata for {len(resume_df)} resumes")
except Exception as e:
    print(f"Warning: Could not load CSV file. Will proceed without metadata: {e}")
    resume_df = None

# List to store all loaded documents
all_documents = []

# Check if the data directory exists
if not os.path.exists(DATA_DIR):
    print(f"Warning: Data directory '{DATA_DIR}' not found. Please check the path.")
else:
    # Get all categories (subdirectories)
    categories = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
    
    for category in categories:
        category_path = os.path.join(DATA_DIR, category)
        print(f"Loading resumes from category: {category}")
        
        # Use DirectoryLoader to load all PDFs in the category directory
        loader = DirectoryLoader(
            category_path, 
            glob="**/*.pdf",  # Load all PDFs, including in subdirectories
            loader_cls=PyPDFLoader
        )
        
        try:
            docs = loader.load()
            # Add metadata: include the category and filename
            for doc in docs:
                doc.metadata["category"] = category
                filename = os.path.basename(doc.metadata["source"])
                doc.metadata["file_name"] = filename
                doc.metadata["id"] = os.path.splitext(filename)[0]  # Remove extension to get ID
                
                # Add additional metadata from CSV if available
                if resume_df is not None:
                    resume_id = doc.metadata["id"]
                    resume_info = resume_df[resume_df["ID"] == resume_id]
                    if not resume_info.empty:
                        # Add any additional metadata from the CSV
                        pass
            
            all_documents.extend(docs)
            print(f"  Loaded {len(docs)} resumes from {category}")
        except Exception as e:
            print(f"  Error loading documents from {category}: {e}")

print(f"Total resumes loaded: {len(all_documents)}")

Loaded metadata for 2484 resumes
Loading resumes from category: DESIGNER
  Loaded 202 resumes from DESIGNER
Loading resumes from category: BPO
  Loaded 47 resumes from BPO
Loading resumes from category: FINANCE
  Loaded 238 resumes from FINANCE
Loading resumes from category: CONSTRUCTION
  Loaded 228 resumes from CONSTRUCTION
Loading resumes from category: SALES
  Loaded 205 resumes from SALES
Loading resumes from category: AUTOMOBILE
  Loaded 72 resumes from AUTOMOBILE
Loading resumes from category: CONSULTANT
  Loaded 236 resumes from CONSULTANT
Loading resumes from category: CHEF
  Loaded 230 resumes from CHEF
Loading resumes from category: APPAREL
  Loaded 188 resumes from APPAREL
Loading resumes from category: AGRICULTURE
  Loaded 132 resumes from AGRICULTURE
Loading resumes from category: TEACHER
  Loaded 185 resumes from TEACHER
Loading resumes from category: HR
  Loaded 225 resumes from HR
Loading resumes from category: DIGITAL-MEDIA
  Loaded 180 resumes from DIGITAL-MEDIA
Load

In [4]:
##########################
# 2. Split Documents into Chunks
##########################

# Define the embedding model name (also used for tokenization)
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

# Define a list of Markdown separators (from LangChain's MarkdownTextSplitter)
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

def split_documents(chunk_size: int, knowledge_base: list, tokenizer_name: str = EMBEDDING_MODEL_NAME):
    """
    Splits documents into chunks using a Hugging Face tokenizer.
    Adds overlap and uses custom Markdown separators.
    Removes duplicate chunks.
    """
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )
    
    docs_processed = []
    for doc in knowledge_base:
        docs_processed.extend(text_splitter.split_documents([doc]))
    
    # Remove duplicates based on content
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)
    return docs_processed_unique

# Split the loaded documents into chunks (adjust chunk_size as needed)
docs_processed = split_documents(512, all_documents, tokenizer_name=EMBEDDING_MODEL_NAME)
print(f"Total chunks after splitting: {len(docs_processed)}")

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Total chunks after splitting: 11255


In [5]:
##########################
# 3. Build a FAISS Vector Index
##########################

# Initialize HuggingFaceEmbeddings (using GPU if available)
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"} if torch.cuda.is_available() else {"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},  # Use cosine similarity
)

# Create FAISS vector store from the processed document chunks
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/66.7M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chunks:   0%|          | 0/10 [00:00<?, ?it/s]

In [6]:
##########################
# 4. Set Up the Open-Source LLM for Answer Generation
##########################

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
##########################
# 5. Define the Chat-Style Prompt Template for Resume Analysis
##########################

prompt_in_chat_format = [
    {
        "role": "system",
        "content": (
            "You are a resume analysis assistant. Using only the information contained in the provided resume segments, "
            "answer the user's question accurately and concisely.\n\n"
            "- Strictly refer to the provided context—do not generate or assume information.\n"
            "- Include the source document number and category when applicable.\n"
            "- If the context does not provide enough information, explicitly state: 'The answer cannot be determined from the provided resume segments.'\n"
            "- Ensure neutrality and objectivity in your analysis."
        ),
    },
    {
        "role": "user",
        "content": (
            "Resume segments:\n{context}\n---\nNow, answer the following question based on the provided resume segments.\n\n"
            "Question: {question}"
        ),
    },
]

# Use the tokenizer's helper to apply the chat template
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)

##########################
# 6. Initialize the Reranker
##########################

# Load a reranker model (e.g., from ColBERT v2)
RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [8]:
##########################
# 7. Define the RAG Function with Re-Ranking for Resume Analysis
##########################

def answer_with_rag(
    question: str,
    llm: pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
    category_filter: Optional[str] = None,
) -> Tuple[str, List[str], List[dict]]:
    print("=> Retrieving resume segments...")
    
    # Retrieve a larger set of candidate documents
    # If a category filter is provided, we'll filter results post-retrieval
    retrieved_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    
    # Apply category filter if specified
    if category_filter:
        retrieved_docs = [doc for doc in retrieved_docs if doc.metadata.get("category") == category_filter]
        # If filtering reduced docs below our final number, adjust
        num_docs_final = min(num_docs_final, len(retrieved_docs))
    
    retrieved_texts = [doc.page_content for doc in retrieved_docs]
    retrieved_metadata = [doc.metadata for doc in retrieved_docs]
    
    # Optionally rerank the retrieved documents
    if reranker and len(retrieved_docs) > num_docs_final:
        print("=> Reranking resume segments...")
        reranked = reranker.rerank(question, retrieved_texts, k=num_docs_final)
        
        # Reranker returns a list of dicts with "content" and indices
        final_texts = [item["content"] for item in reranked]
        
        # Get the corresponding metadata for reranked documents
        final_metadata = []
        for item in reranked:
            if "index" in item:
                final_metadata.append(retrieved_metadata[item["index"]])
            else:
                # If index not available, try to match by content
                idx = retrieved_texts.index(item["content"]) if item["content"] in retrieved_texts else None
                if idx is not None:
                    final_metadata.append(retrieved_metadata[idx])
                else:
                    # Fallback if we can't match
                    final_metadata.append({})
    else:
        final_texts = retrieved_texts[:num_docs_final]
        final_metadata = retrieved_metadata[:num_docs_final]
    
    # Build the final context string from the top documents
    context = "\nResume segments:\n"
    for i, (text, metadata) in enumerate(zip(final_texts, final_metadata)):
        category = metadata.get("category", "Unknown")
        filename = metadata.get("file_name", "Unknown")
        context += f"Document {i} (Category: {category}, File: {filename}):::\n{text}\n\n"
    
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)
    print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]
    print("Answer Generated!!!")
    return answer, final_texts, final_metadata

##########################
# 8. Query Function with Optional Category Filter
##########################

def query_resumes(user_query, category=None):
    """
    Query the resume database with an option to filter by job category.
    
    Args:
        user_query (str): User's question about resumes
        category (str, optional): Category to filter by (e.g., "HR", "Designer")
        
    Returns:
        dict: Contains the answer and metadata about retrieved documents
    """
    # Generate the answer using the RAG function with re-ranking
    answer, final_docs, final_metadata = answer_with_rag(
        user_query, 
        READER_LLM, 
        KNOWLEDGE_VECTOR_DATABASE, 
        reranker=RERANKER,
        category_filter=category
    )
    
    # Format results
    result = {
        "answer": answer,
        "retrieved_docs": []
    }
    
    # Add retrieved document info
    for i, (doc, metadata) in enumerate(zip(final_docs, final_metadata)):
        doc_info = {
            "index": i,
            "category": metadata.get("category", "Unknown"),
            "file_name": metadata.get("file_name", "Unknown"),
            "id": metadata.get("id", "Unknown"),
            "content_preview": doc[:200] + "..." if len(doc) > 200 else doc
        }
        result["retrieved_docs"].append(doc_info)
        
    return result

In [9]:
##########################
# 9. Sample Usage
##########################

# Example queries:
sample_queries = [
    "What skills are common in IT resumes?",
    "Compare the education backgrounds in HR vs Finance resumes",
    "What certifications are popular in Healthcare resumes?",
    "Find resumes with experience in project management",
    "What are common job titles in Business Development resumes?"
]

# Example usage:
if len(all_documents) > 0:
    print("\n==================================Sample Query==================================\n")
    # Example: Query with category filter
    sample_query = "What skills are commonly mentioned in these resumes?"
    category_filter = None  # Set to a specific category like "Information-Technology" or None for all
    
    result = query_resumes(sample_query, category=category_filter)
    
    print(f"Question: {sample_query}")
    if category_filter:
        print(f"Category Filter: {category_filter}")
    
    print("\n==================================Answer==================================\n")
    print(result["answer"])
    
    print("\n==================================Retrieved Document Info==================================")
    for doc in result["retrieved_docs"]:
        print(f"\nDocument {doc['index']}:")
        print(f"  Category: {doc['category']}")
        print(f"  File: {doc['file_name']}")
        print(f"  ID: {doc['id']}")
        print(f"  Preview: {doc['conte nt_preview']}")
else:
    print("No documents were loaded. Please check the data directory path.")


==================================Sample Query==================================

=> Retrieving resume segments...


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

=> Reranking resume segments...


/usr/local/lib/python3.10/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
100%|██████████| 1/1 [00:00<00:00,  4.85it/s]


=> Generating answer...
Answer Generated!!!
Question: What skills are commonly mentioned in these resumes?

==================================Answer==================================

Some commonly mentioned skills across all the resumes include:

1. Multi-tasking
2. Communication skills (verbal and written)
3. Problem solving
4. Time management
5. Leadership
6. Customer service
7. Organization skills
8. Microsoft Office proficiency (specifically Excel, PowerPoint, and Word)
9. Email and phone skills
10. Team building
11. Sales experience
12. Ability to work well with others
13. Project management skills
14. Problem identification and resolution
15. Attention to detail
16. Familiarity with accounting systems and principles
17. Familiarity with aviation policies and procedures
18. Familiarity with fitness promotion and goal setting
19. Familiarity with HR practices such as employee relations and handbook creation

Note that this list is not exhaustive and may vary depending on the speci

In [17]:
##########################
# 10. Save the Vector Store for Future Use
##########################

# Save the FAISS index
KNOWLEDGE_VECTOR_DATABASE.save_local("resume_vector_store")
print("\nSaved vector store to 'resume_vector_store'. You can load it later with:")
print("from langchain.vectorstores import FAISS")
print("from langchain_huggingface import HuggingFaceEmbeddings")
print("embedding_model = HuggingFaceEmbeddings(model_name='thenlper/gte-small')")
print("vector_store = FAISS.load_local('resume_vector_store', embedding_model)")


Saved vector store to 'resume_vector_store'. You can load it later with:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name='thenlper/gte-small')
vector_store = FAISS.load_local('resume_vector_store', embedding_model)


In [15]:
sample_query = """
The following is the job description: 
'Job Title: Business Development Consultant

Job Duties:

Developing connections and new sources of business and relationship management with existing accounts
Networking and building relationships with business partners and jobseekers
Identifying and pursuing new business opportunities to expand the client base
Managing key accounts to ensure the delivery standard and fulfilment level
Provide financial analysis and modeling to support business cases.
Business partnering with internal stakeholders to improve current processes and implement new procedures
Conducting market research and competitive analysis to identify trends and opportunities for growth
Consult with clients to understand their needs and provide tailored solutions.
Preparing and delivering presentations and proposals to potential clients
Developing strategies for brand promotion and lead generation
Other ad-hoc duties apply


Requirements:

Minimum Diploma / Degree in Business Administration of Management
At least 2 year of Sales / Business development experience
Good communication and interpersonal skills
Excellent time management and organizational skills
The potential candidate must be motivated to stay ahead by keeping up with the latest technological advancements and market trends.'

Your tasks are as follows:

1. Analyze all the resumes to assess its alignment with the provided job description.
2. Stick strictly to the facts provided in the resumes and job description. Never make assumptions or invent information.
3. Select only the top 3 resumes based on their alignment with the job description. Ensure these are ranked clearly by relevance (1st: Most Suitable, 2nd: Next Best, 3rd: Final Option).
4. Evaluate key areas, including:
    Education: Does the candidate meet the educational requirements?
    Experience: Is their work experience relevant and sufficient for the role?
    Skills: Are the technical, soft, and domain-specific skills present?
5. For each resume, provide:
    Strengths: Key qualifications that match the role.
    Gaps: Areas where the candidate does not meet the criteria.
6. Summarize why these top 3 resumes were chosen, focusing on their alignment with the job requirements.
"""

In [16]:

# Example usage:
if len(all_documents) > 0:
    print("\n==================================Sample Query==================================\n")
    # Example: Query with category filter
    # sample_query = "What skills are commonly mentioned in these resumes?"
    category_filter = None  # Set to a specific category like "Information-Technology" or None for all
    
    result = query_resumes(sample_query, category=category_filter)
    
    print(f"Question: {sample_query}")
    if category_filter:
        print(f"Category Filter: {category_filter}")
    
    print("\n==================================Answer==================================\n")
    print(result["answer"])
    
    print("\n==================================Retrieved Document Info==================================")
    for doc in result["retrieved_docs"]:
        print(f"\nDocument {doc['index']}:")
        print(f"  Category: {doc['category']}")
        print(f"  File: {doc['file_name']}")
        print(f"  ID: {doc['id']}")
        print(f"  Preview: {doc['content_preview']}")
else:
    print("No documents were loaded. Please check the data directory path.")


==================================Sample Query==================================

=> Retrieving resume segments...


Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

=> Reranking resume segments...


100%|██████████| 1/1 [00:00<00:00,  2.66it/s]


=> Generating answer...
Answer Generated!!!
Question: 
The following is the job description: 
'Job Title: Business Development Consultant

Job Duties:

Developing connections and new sources of business and relationship management with existing accounts
Networking and building relationships with business partners and jobseekers
Identifying and pursuing new business opportunities to expand the client base
Managing key accounts to ensure the delivery standard and fulfilment level
Provide financial analysis and modeling to support business cases.
Business partnering with internal stakeholders to improve current processes and implement new procedures
Conducting market research and competitive analysis to identify trends and opportunities for growth
Consult with clients to understand their needs and provide tailored solutions.
Preparing and delivering presentations and proposals to potential clients
Developing strategies for brand promotion and lead generation
Other ad-hoc duties apply


Re